### 引入库

In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

#### 参数设置

In [2]:
n_epochs = 3          #学习次数，根据拟合程度调整。太大过拟合；太小欠拟合
batch_size_train = 100 #训练时的批次大小，根据拟合程度和硬件性能调整。太大过拟合，占用内存大，训练速度慢；太小易受噪声影响，易陷入局部最小值
batch_size_test = 50  #测试时的批次大小，调整没啥影响，一般和训练批次大小相似
learning_rate = 0.01  #学习率，控制参数调整幅度。太大不稳定，可能跳过全局最小值；太小收敛速度慢，易陷入局部最小值
momentum = 0.5        #动量，减少陷入局部最小值情况。太大不稳定，可能跳过全局最小值；太小收敛速度慢，易陷入局部最小值
log_interval = 100     #日志输出间隔，控制输出训练信息频率

random_seed = 1 #随机种子，用于复现结果
torch.manual_seed(random_seed) #设置随机种子

# 检查是否有可用的GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

#### 导入MNIST数据集  
自动下载到根目录下的 ./data/ 路径

数据集保存在 train_loader 和 test_loader 中
每张图片的灰度值 tensor 位于 train_loader.dataset.data 和 train_loader.dataset.data 中 
数量分别为 6w 和 1w

每张图片的 tensor 
dtype=torch.uint8
torch.Size([28, 28])

In [3]:
train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('./data/', #数据集本地存储路径，没有则创建
                               train=True, #加载训练集
                               download=True, #如果本地不存在数据集，会下载数据集
                               transform=torchvision.transforms.ToTensor(), #将图像转换为PyTorch张量                          
                               ),
    batch_size=batch_size_train, #设置批次大小
    shuffle=True,  #每次打乱
    )   

test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('./data/',
                               train=False,
                               download=True,
                               transform=torchvision.transforms.Compose([
                                   torchvision.transforms.ToTensor(),
                                   ]),
                               ),
    batch_size=batch_size_test,
    shuffle=True,
    )

#### 定义卷积网络

In [4]:
#定义神经网络
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__() #初始化方法
        self.conv1 = nn.Conv2d(1, 15, kernel_size=5) #卷积层1：输入通道数，输出通道数，kernel_size表示卷积核的大小
        self.conv2 = nn.Conv2d(15, 20, kernel_size=5) #卷积层2
        self.fc1 = nn.Linear(320, 50) #全连接层类1:输入特征数，输出特征数
        self.fc2 = nn.Linear(50, 10) #全连接层类2
 
    def forward(self, x):     #前向传播方法
        x = F.relu(F.max_pool2d(self.conv1(x), 2))         
        x = F.relu(F.max_pool2d((self.conv2(x)), 2))
        x = x.view(-1, 320)                      #将二维张量x展平为一维张量。使用-1可以根据其他维度推断出一个维度的大小，320是展平后张量的大小。
        x = F.relu(self.fc1(x))                  #对展平后的张量应用全连接层1，对全连接层1的结果应用ReLU激活函数
        x = self.fc2(x)                          #对张量x应用全连接层2
        return F.log_softmax(x, dim=1) #对第二个全连接层的输出应用对数softmax函数，获取对数概率；dim=1指定应在列上应用，该维度表示不同的类别

network = Net().to(device)